<a href="https://colab.research.google.com/github/soupsoupr/StrokeSegmentation/blob/main/testOnCNNsConstruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import nibabel as nib
import os
# Set the file path to the directory containing the DWI data files
file_path = '/content/drive/MyDrive/ISLES/dataset-ISLES22^public^unzipped^version'

for example_case in range(1,11): # for the first 10 data
    # Set images path.
    dwi_path = os.path.join(file_path, 'rawdata', 'sub-strokecase{}'.format("%04d" %example_case), 'ses-0001', 
                            'sub-strokecase{}_ses-0001_dwi.nii.gz'.format("%04d" %example_case))
    mask_path = dwi_path.replace('rawdata', 'derivatives').replace('dwi', 'msk')

# Load image data.
dwi_data = []
mask_data = []

dwi_data = nib.load(dwi_path).get_fdata()
mask_data = nib.load(mask_path).get_fdata()

In [ ]:
print(dwi_data.shape)
print(mask_data.shape)

# Construction CNNs model

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

## Reshape dwi data

In [ ]:
# Reshape the data to be suitable for a CNN model
dwi_data = np.reshape(dwi_data, (112, 112, 72, 1))
mask_data = np.reshape(mask_data, (112, 112, 72, 1))

## Split DWI data into train and test set

In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(dwi_data, mask_data, test_size=0.2, random_state=42)

# Define input shape

In [ ]:
# Define the input shape
input_shape = train_data.shape

print(input_shape)

## define the model architecture

In [ ]:
# Define the model architecture
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(112, 112, 72)))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

## Complie the model

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

## Train the model

In [ ]:
# Train the model
model.fit(x=train_data, y=train_labels, epochs=5, batch_size=16, validation_split=0.2)